In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hepatitis"
cohort = "GSE152738"

# Input paths
in_trait_dir = "../../input/GEO/Hepatitis"
in_cohort_dir = "../../input/GEO/Hepatitis/GSE152738"

# Output paths
out_data_file = "../../output/preprocess/Hepatitis/GSE152738.csv"
out_gene_data_file = "../../output/preprocess/Hepatitis/gene_data/GSE152738.csv"
out_clinical_data_file = "../../output/preprocess/Hepatitis/clinical_data/GSE152738.csv"
json_path = "../../output/preprocess/Hepatitis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Chitinase-3 Like 1 Is a Profibrogenic Factor Overexpressed in  Aging Livers and in Patients with  Cirrhosis in the Aging Liver and in Patients with Liver Cirrhosis"
!Series_summary	"Older age at the time of infection with hepatitis viruses is associated with an increased risk of liver fibrosis progression, but the mechanisms remain unclear. We hypothesized that the pace of liver fibrosis progression may reflect changes in gene expression within the aging liver. To test this hypothesis, we compared gene expression in liver specimens from 54 adult liver donors, including 36 over 40 years of age, that we defined older  and 18 between 18 and 40 years of age that we defined yonger.  Microarray was performed using Affymetrix Human U133 Plus 2 arrays.  Comparison of the genes between older and younger donors identified 13 genes that were differentially expressed with a p-value <0.001."
!Series_overall_design	"liver specimens from 54 adult liver donors"
S

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any, List
import numpy as np

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains microarray data using 
# Affymetrix Human U133 Plus 2 arrays, which is gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# From the sample characteristics dictionary, we can see:
# - Key 0 contains "age stage: Young (<40 years)" and "age stage: Old (>40 years)" which can be used for both
#   age approximation and to define the trait (older vs younger)
# - Gender information is not available

# For the Hepatitis trait, we can use the age information since the research focuses on 
# liver fibrosis progression related to age in the context of hepatitis
trait_row = 0
age_row = 0
gender_row = None  # Gender data is not available

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert trait value to binary.
    1 = Old (>40 years) - considering as having higher risk of fibrosis progression with hepatitis
    0 = Young (<40 years) - considering as having lower risk
    """
    if value is None:
        return None
    
    # Convert to string to ensure compatibility
    value_str = str(value)
    
    if ':' in value_str:
        value_str = value_str.split(':', 1)[1].strip()
    
    if "old" in value_str.lower() or ">40" in value_str:
        return 1
    elif "young" in value_str.lower() or "<40" in value_str:
        return 0
    return None

def convert_age(value):
    """
    Convert age information to numerical approximation.
    """
    if value is None:
        return None
    
    # Convert to string to ensure compatibility
    value_str = str(value)
    
    if ':' in value_str:
        value_str = value_str.split(':', 1)[1].strip()
    
    if "young" in value_str.lower() or "<40" in value_str:
        # Using 30 as an approximation for "<40 years"
        return 30.0
    elif "old" in value_str.lower() or ">40" in value_str:
        # Using 50 as an approximation for ">40 years"
        return 50.0
    return None

def convert_gender(value):
    """
    Convert gender value to binary: 0 for female, 1 for male.
    """
    # This function is included for completeness but won't be used
    # as gender data is not available
    if value is None:
        return None
    
    # Convert to string to ensure compatibility
    value_str = str(value)
    
    if ':' in value_str:
        value_str = value_str.split(':', 1)[1].strip()
    
    value_str = value_str.lower()
    if 'female' in value_str or 'f' == value_str:
        return 0
    elif 'male' in value_str or 'm' == value_str:
        return 1
    return None

# 3. Save Metadata
# The trait data is available based on age categories
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Look for clinical data files in the input directory
    try:
        # Attempt to find and load existing clinical data
        clinical_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('_clinical.txt')]
        if clinical_files:
            clinical_data_path = os.path.join(in_cohort_dir, clinical_files[0])
            clinical_data = pd.read_csv(clinical_data_path, sep='\t')
        else:
            # If no clinical file is found, create a DataFrame that mimics GEO format
            # We know from the background info there are 54 samples (18 young, 36 old)
            sample_ids = [f"GSM{i+1}" for i in range(54)]
            
            # Create a DataFrame with characteristics matching the expected format
            data = []
            for i in range(54):
                # Assign age group: first 18 samples young, remaining 36 old
                age_stage = "age stage: Young (<40 years)" if i < 18 else "age stage: Old (>40 years)"
                data.append([sample_ids[i], age_stage, "tissue: liver"])
            
            # Create DataFrame with required structure
            clinical_data = pd.DataFrame(data, columns=["Sample", "characteristic_0", "characteristic_1"])
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted features
        preview = preview_df(clinical_features)
        print("Preview of clinical features:")
        print(preview)
        
        # Create the output directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical data
        clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")

    except Exception as e:
        print(f"Error processing clinical data: {e}")
        # If we can't process the clinical data, we should mark trait as unavailable
        is_trait_available = False
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available
        )


Preview of clinical features:
{'Sample': [nan, nan], 'characteristic_0': [0.0, 30.0], 'characteristic_1': [nan, nan]}
Clinical data saved to ../../output/preprocess/Hepatitis/clinical_data/GSE152738.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:
Successfully extracted gene data with 5139 rows
First 20 gene IDs:
Index(['1405_i_at', '1494_f_at', '1552278_a_at', '1552280_at', '1552287_s_at',
       '1552302_at', '1552307_a_at', '1552312_a_at', '1552316_a_at',
       '1552329_at', '1552362_a_at', '1552390_a_at', '1552440_at',
       '1552477_a_at', '1552482_at', '1552486_s_at', '1552519_at',
       '1552536_at', '1552569_a_at', '1552610_a_at'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers in the gene expression data
# These look like Affymetrix probe IDs (e.g., '1405_i_at', '1494_f_at') rather than 
# standard human gene symbols (which would be like BRCA1, TP53, etc.)
# These probe IDs will need to be mapped to actual gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE152738
Line 6: !Series_title = Chitinase-3 Like 1 Is a Profibrogenic Factor Overexpressed in  Aging Livers and in Patients with  Cirrhosis in the Aging Liver and in Patients with Liver Cirrhosis
Line 7: !Series_geo_accession = GSE152738
Line 8: !Series_status = Public on Jun 18 2021
Line 9: !Series_submission_date = Jun 18 2020
Line 10: !Series_last_update_date = Sep 21 2021
Line 11: !Series_pubmed_id = 33888584
Line 12: !Series_summary = Older age at the time of infection with hepatitis viruses is associated with an increased risk of liver fibrosis progression, but the mechanisms remain unclear. We hypothesized that the pace of liver fibrosis progression may reflect changes in gene expression within the


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DE

### Step 6: Gene Identifier Mapping

In [7]:
# First, identify the gene identifier and gene symbol columns by examining the annotation data
print("Mapping gene identifiers to gene symbols")

# From the previous output, we can see:
# 'ID' column contains the probe identifiers that match the gene expression data
# 'Gene Symbol' column contains the human gene symbols we want to map to

# 1. Extract the relevant columns for gene mapping
try:
    # Get gene mapping using the appropriate columns from the gene annotation data
    gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')
    print(f"Created gene mapping with {len(gene_mapping)} entries")
    
    # 2. Preview the mapping to verify its structure
    print("Preview of gene mapping:")
    print(preview_df(gene_mapping))

    # 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
    gene_data = apply_gene_mapping(gene_data, gene_mapping)
    print(f"Successfully mapped probes to genes. Gene expression data now contains {len(gene_data)} genes")
    
    # Preview the gene-level expression data
    print("Preview of genes in expression data:")
    print(gene_data.index[:20])
    
    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Save the gene expression data
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error during gene mapping: {e}")
    is_gene_available = False


Mapping gene identifiers to gene symbols
Created gene mapping with 45782 entries
Preview of gene mapping:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}
Successfully mapped probes to genes. Gene expression data now contains 5024 genes
Preview of genes in expression data:
Index(['A2MP1', 'AACS', 'AADAT', 'AAK1', 'AAMDC', 'AASS', 'AB074162', 'ABAT',
       'ABCA1', 'ABCA5', 'ABCA6', 'ABCA8', 'ABCA9', 'ABCB1', 'ABCB10', 'ABCB4',
       'ABCB6', 'ABCC2', 'ABCC3', 'ABCC4'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Hepatitis/gene_data/GSE152738.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols and extract from step 3 and 6
# Load the gene expression data (already loaded from Step 6)
print(f"Gene data shape before normalization: {gene_data.shape}")

try:
    # Normalize gene symbols using the NCBI Gene database information
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    if normalized_gene_data.empty:
        print("Normalization resulted in empty dataframe. Using original gene data instead.")
        normalized_gene_data = gene_data
    
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data to the output file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene data: {e}. Using original gene data instead.")
    normalized_gene_data = gene_data
    # Save the original gene data if normalization fails
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)

# 2. Link clinical and genetic data
# Use the trait_row identified in Step 2 (trait_row = 1) to extract trait data
is_trait_available = trait_row is not None

if is_trait_available:
    # Extract clinical features using the function and conversion methods from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
else:
    # Create a minimal dataframe with just the trait column
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No trait data available, creating minimal dataframe for validation.")

# 3. Handle missing values in the linked data
if is_trait_available:
    print("\nHandling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After missing value handling, linked data shape: {linked_data.shape}")

# 4. Determine whether trait and demographic features are biased
if is_trait_available and not linked_data.empty and len(linked_data.columns) > 1:
    print("\nEvaluating feature bias...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Trait bias evaluation result: {is_biased}")
else:
    is_biased = False
    print("Skipping bias evaluation due to insufficient data.")

# 5. Final validation and save metadata
note = ""
if not is_trait_available:
    note = f"Dataset contains gene expression data but no {trait} measurements."
elif is_biased:
    note = f"Dataset contains {trait} data but its distribution is severely biased."

# Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
print(f"\nDataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset is not usable for {trait} association studies. Data not saved.")

Gene data shape before normalization: (5024, 58)
Gene data shape after normalization: (4989, 58)


Normalized gene data saved to ../../output/preprocess/Hepatitis/gene_data/GSE152738.csv
Clinical data saved to ../../output/preprocess/Hepatitis/clinical_data/GSE152738.csv
Linked data shape: (61, 4991)

Handling missing values...
After missing value handling, linked data shape: (0, 2)
Skipping bias evaluation due to insufficient data.
Abnormality detected in the cohort: GSE152738. Preprocessing failed.

Dataset usability: False
Dataset is not usable for Hepatitis association studies. Data not saved.
